In [1]:
from CC import CC
from DnsVhost import DnsVhost
from MariaDB import MariaDB
from WP import WP
from Partition import Partition
from SMB import SMB
from NFS import NFS
from Diskquota import Diskquota
from ISCSI import ISCSI
from FTP import FTP
from RAID import RAID
from Tomcat import Tomcat
from Nginx import Nginx
from SNMP import SNMP
from Cacti import Cacti
from JL import JL

#---------------------------------------#
#             SSH 객체  생성             #
#---------------------------------------#
server_ip = "172.16.16.24"
server_account = "root"
server_passwd = "asd123!@"
cc =  CC(f"{server_ip}", f"{server_account}", f"{server_passwd}")
cc.connect()

#---------------------------------------#
#         여러 클래스 객체 생성            #
#---------------------------------------#

os = input("현재 OS를 입력해주세요. ex. Rocky, Ubuntu : ")

dv = DnsVhost(os) 
mariadb = MariaDB(os)
wp = WP(os)
part = Partition(os)
smb = SMB(os)
nfs = NFS(os)
dq = Diskquota(os)
ftp = FTP(os)
raid = RAID(os)
tomcat = Tomcat(os)
nginx = Nginx(os)
snmp = SNMP(os)
cacti = Cacti(os)
jl = JL(os)

#---------------------------------------#
#                 전역변수               #
#---------------------------------------#
ZONEFILE = ""


###################################################
#              HTTP,DNS -DnsVhost.py              #
###################################################
## 1. http, DNS 설치 및 초기 세팅 
def install_and_setting_http_dns() :
    print("[HTTP, DNS] 설치 시작합니다.")
    #========= HTTP ========#
    cc.run(dv.install_http())
    dv.set_servername()
    cc.run(dv.mk_vhostconf())
    cc.run(dv.test_vhost())
    # web_dir,third_domain,error_log ,custom_log
    cc.run(dv.set_vhostconf("/home/test","test"))

    #========= DNS ========#
    cc.run(dv.install_named())
    cc.run(dv.init_set_named_conf())
    cc.run(dv.init_set_named_zone())
    cc.run(dv.mk_zone_file())
    cc.run(dv.init_set_zone_file())
    cc.run(dv.set_dns_Third_domain("test"))

    #======= RESTART ========#
    cc.run(dv.restart_http())
    cc.run(dv.restart_dns())
    print("[HTTP, DNS] 설치 및 설정 완료")
    
## 2. 초기 세팅 이후에 새롭게 DnsVhost클래스 사용할 경우에 해당됨 (기본)
def mod_http_dns(web_dir,third_domain):
    print("[HTTP, DNS] 추가 시작")
    dv.set_servername()
    #set_vhostconf(self,web_dir,third_domain):
    cc.run(dv.set_vhostconf(web_dir,third_domain))
    cc.run(f"mkdir -p {web_dir} && chmod 777 {web_dir}")
    cc.run(dv.set_dns_Third_domain(third_domain))
    cc.run(dv.restart_dns())
    cc.run(dv.restart_http())
    print("[HTTP, DNS] 추가 완료")

## 3. 초기 세팅 이후에 새롭게 DnsVhost클래스 사용할 경우에 해당됨 (리버시 프록시)
def mod_http_dns_reverse_proxy(third_domain,port_num):
    print("[HTTP, DNS] 추가 시작 <리버시 프록시>")
    dv.set_servername()
    #set_reverse_proxy_vhostconf(self,third_domain,port_num)
    cc.run(dv.set_reverse_proxy_vhostconf(third_domain,port_num))
    cc.run(dv.set_dns_Third_domain(third_domain))
    cc.run(dv.restart_dns())
    cc.run(dv.restart_http())
    print("[HTTP, DNS] 추가 완료 <리버시 프록시>")
#---------------------실행부-----------------------#
#install_and_setting_http_dns()
#mod_http_dns("/home/test2/","test2")



###################################################
#            MariaDB,PHP -MariaDB.py              #
###################################################
## 1. mariaDB와 php 둘 다 설치함
def install_mariaDB_and_php():
    print("[mariaDB] 설치 시작합니다.")
    cc.run(mariadb.install_packages())
    cc.run(mariadb.install_apache_and_php())
    cc.run(mariadb.install_mariaDB())
    cc.run(mariadb.init_mariaDB())
    cc.run(mariadb.install_php_my_admin())
    cc.run(mariadb.set_phpMyadmin_conf())
    print("[mariaDB] 설치 완료")
#---------------------실행부-----------------------#
#install_mariaDB_and_php()


##################################################
#           Partition -Partition.py              #
##################################################
def mk_partition():
    print("[partition] 시작합니다.")
    cc.run("lsblk")
    cc.run(part.make_partition())
    cc.run(part.format_partition())
    cc.run(part.mount())
    cc.run(part.permanent_mount_config_remote())
    print("[partition] 완료")

#---------------------실행부-----------------------#
#mk_partition()

###################################################
#                    SMB -SMB.py                  #
###################################################

def install_smb():
    cc.run(smb.smb_install())
    cc.run(smb.smb_intsll_status_check())   
    choose = input("smb 서버와 클라이언트 중 선택하세요.(S/C) ex)S : ")
    if(choose == "S" ) :
        server_smb()
    if(choose == "C" ) :
        client_smb()

def server_smb():
    print("[SMB] SERVER 시작")
    cc.run(smb.smb_share_folder_make())
    cc.run(smb.smb_mk_smb_id())
    cc.run(smb.smb_set_smb_id_passwd())
    cc.run(smb.smb_id_check())
    cc.run(smb.smb_share_folder_authority())
    cc.run(smb.smb_mk_back())
    cc.run(smb.smb_set())
    cc.run(smb.smb_server_start())
    print("[SMB] SERVER 설정완료")    
def client_smb() :
    print("[SMB] CLIENT 시작")
    cc.run(smb.smb_client_login())
    cc.run(smb.smb_client_mount())
    print("[SMB] CLIENT 완료")
#---------------------실행부-----------------------#
install_smb()

###################################################
#                    NFS -NFS.py                  #
###################################################

def install_nfs():
    cc.run(nfs.nfs_isntall())
    choose = input("NFS 서버와 클라이언트 중 선택하세요.(S/C) ex)S : ")
    if(choose == "S" ) :
        server_nfs()
    if(choose == "C" ) :
        client_nfs()

def server_nfs():
    cc.run(nfs.nfs_share_folder_make())
    cc.run(nfs.nfs_share_folder_authority())
    cc.run(nfs.server_nfs_set_exports_config())
    cc.run(nfs.server_nfs_start())

def client_nfs():
    cc.run(nfs.nfs_client_mount_temporary())
    cc.run(nfs.nfs_clinet_mount_permanent())
#---------------------실행부-----------------------#
#install_nfs()

###################################################
#            Diskquota -diskquota.py              #
###################################################
def diskquota_install_and_set():
    cc.run(dq.diskquota_isntall())
    cc.run(dq.diskquota_install_check())
    cc.run("lsblk")
    print("TYPE MOUNTPOINTS가 part로 되어있는 디스크만 가능합니다. 안되면 partition을 해야합니다")
    exist_part = input("part로 되어있는 디스크가 없으시거나 해당 디스크가 마운트가 안되어있는 경우에 반드시 n 입력해주세요.\n 있으시면 y 또는 아무문자열 입력 ex) y: ")
    if(exist_part == "n") :
        return
    cc.run(dq.diskquota_modify())
    cc.run(dq.diskquota_mount_and_initalization())
    cc.run(dq.diskquota_settings())
    cc.run(dq.diskquota_systemctl())
    
#---------------------실행부-----------------------#
#diskquota_install_and_set()



###################################################
#                  ISCSI -ISCSI.py                #
###################################################
def iscsi_all_in_one():
    client_os = input("클라이언의 OS를 입력하세요 ex. Rocky, Ubuntu : ")
    iscsi = ISCSI(os,client_os)

    #=========== server 영역 ===========#
    #cc.run(iscsi.set_firewall())
    cc.run(iscsi.install_target())
    
    #=========== client 영역 ===========#
    client_ip = input("새로운 SSH 연결을 합니다. 클라이언트 IP를 입력하세요")
    client_account = input("클라이언트 계정 입력하세요 ex. root")
    client_passwd = input("클라이언트 계정의 비밀번호를 입력하세요 ex. asd123!@")
    cc_client =  CC(f"{client_ip}", f"{client_account}", f"{client_passwd}")
    cc_client.connect()
    
    cc_client.run(iscsi.iscsi_client_install())
    cc_client.run(iscsi.iscsi_client_iqn())
    iscsi_client_iqn_domain = input("위에서 확인 CLIENT의 IQN을 복사해주세요")

    #=========== server 영역 ===========#
    print("[ISCSI 서버] ISCSI 서버를 세팅해보겠습니다.")
    cc.run("lsblk")
    disk_path = input("위의 결과를 확인하시고 iSCSI로 제공할 디스크 경로를 입력하세요 : ex)/dev/sdb")
    cc.run(iscsi.targetcli(server_ip, iscsi_client_iqn_domain, disk_path))

    #=========== client 영역 ===========#
    print("[ISCSI 클라이언트] ISCSI 클라이언트를 세팅해보겠습니다.")
    cc_client.run(iscsi.iscsi_client_login_set_servers_iscsi(server_ip))
    cc_client.run("iscsiadm -m session")
    print("이제 클라이언트로 돌아가셔서 lsblk로 확인해보세요. 또 iscsiadm -m session로 잘 받았는지 확인해보세요 ^^")
    print("[ISCSI] 모든 설정 및 세팅이 완료되었습니다.")
#---------------------실행부-----------------------#
#iscsi_all_in_one()

###################################################
#                    FTP -FTP.py                  #
###################################################
def inital_FTP():
    print("[FTP 서버] FTP 설치 및 유저를 생성합니다.")
    cc.run(ftp.install_ftp())
    cc.run(ftp.on_off_ftp())
    cc.run(ftp.set_ftp_config())
    cc.run(ftp.add_ftpuser())
    cc.run(ftp.set_ftpuser_passwd())
    cc.run(ftp.set_dir())
    print("[FTP 서버] 설치 및 설정이 완료되었습니다. ")
#---------------------실행부-----------------------#
#inital_FTP()

###################################################
#                  RAID -RAID.py                  #
###################################################
def mk_raid1():
    print("[RAID] RAID1을 만듭니다.")
    cc.run(raid.install_raid())
    cc.run("lsblk")
    cc.run(raid.raid1_mk())
    cc.run(raid.raid1_mk_fs_mount())
    permanent_mount = input("영구마운트 하겠습니까 yes or no (ex. yes) : ")    
    if(permanent_mount == "yes") :
        cc.run(raid.raid1_permanent_mount_config_remote())
    cc.run("lsblk")
    print("[RAID] RAID1 완료")
    
def erase_raid():
    print("[RAID] RAID를 제거합니다.")
    cc.run("lsblk")
    cc.run(raid.erase_raid())
    cc.run("lsblk")
    print("RAID가 제거되었습니다 ^^")
    
#---------------------실행부-----------------------#
#mk_raid1()
#erase_raid()


###################################################
#               Tomcat -Tomcat.py                 #
###################################################
def install_tomcat():
    print("[Tomcat] 설치하겠습니다.")
    cc.run(tomcat.install_java())
    cc.run(tomcat.set_tomcat())
#    mod_http_dns_reverse_proxy(third_domain,"port_num"):
    mod_http_dns_reverse_proxy("tom","8088")
    cc.run(tomcat.set_service())
    print("[Tomcat] 설치 및 설정이 완료되었습니다.")

#---------------------실행부-----------------------#
#install_tomcat()

###################################################
#                 WP,PHPMYSQL -WP.py              #
###################################################
## 1. 로컬) WP 설치 및 wp-config.php 파일 수정까지
def install_wp():
    print("[WP] 설치하겠습니다.")
    print("[전제조건] mariaDB가 있어야하며, 서버팀에게 wp를 위한 유저 및 db를 생성해야합니다.")
    cc.run(wp.install_prev_packages())
    cc.run(wp.install_and_move_dir())
    cc.run(wp.set_wp_config_php_client())
    #def mod_http_dns(web_dir,third_domain):
    mod_http_dns("/home/wp","wp")
    print("[WP] 설치 및 설정이 완료되었습니다.")
#---------------------실행부-----------------------#
#install_wp()

###################################################
#                  Nginx -Nginx.py                #
###################################################
## 1. 로컬) WP 설치 및 wp-config.php 파일 수정까지
def install_nginx():
    print("[Nginx] 설치하겠습니다.")
    print("[전제조건] httpd, dns, php, wp(시범용)가 있어야합니다. ")
    cc.run(nginx.install_nginx())
    servername = input("server name 입력 ex. test.com , ppp.com : ")
    cc.run(nginx.set_vhostconf(servername))
    cc.run(nginx.restart_nginx())
    print("[Nginx] 설치 및 설정이 완료되었습니다.")
#---------------------실행부-----------------------#
#install_nginx()


###################################################
#                    SNMP -SNMP.py                #
###################################################
## 1. 로컬) WP 설치 및 wp-config.php 파일 수정까지
def install_snmp():
    print("[SNMP] 설치하겠습니다.")
    print("[전제조건] Rocky는 dnf있어야함. ")
    cc.run(snmp.install_snmp())
    cc.run(snmp.set_snmp())
    cc.run(snmp.start_enable_snmpd())
    cc.run(snmp.test_snmpd_local())
    print("[SNMP] 설정 완료")
    
#---------------------실행부-----------------------#
#install_snmp()

###################################################
#                  Cacti -Cacti.py                #
###################################################
## 1. 로컬) WP 설치 및 wp-config.php 파일 수정까지
def install_cacti():
    print("[Cacti] 설치하겠습니다.")
    print("[전제조건] MRTG 있어야함. ")
    cc.run(cacti.install_cacti())
    cc.run(cacti.set_mariadb_server_cnf())
    cc.run(cacti.set_cacti())
    cc.run(cacti.set_cacti_db_php())
    cc.run(cacti.set_cacti_conf())
    cc.run(cacti.restart_all())

    print("#======= timezone 세팅 시작합니다. =========#")
    cc.run(cacti.set_my_cnf())
    cc.run(cacti.set_timezone())
    cc.run(cacti.grant_timezone_to_user())
    cc.run(cacti.set_php_ini())
    cc.run(cacti.restart_all())

    print("#======= cron.d 세팅 시작합니다. =========#")
    cc.run(cacti.set_crond())
    
    print("[Cacti] 설정 완료")
    print("이제 브라우저에 접속해주세요.") 
    print("1. [초기 세팅] username : admin , password : admin 입니다.")
    print("2. [Change Password] current password : admin , New password : Asd123!@ 입니다.")
    
#---------------------실행부-----------------------#
#install_cacti()

###################################################
#                    JL -JL.py                    #
###################################################
## 1. 로컬) WP 설치 및 wp-config.php 파일 수정까지
def install_JL():
    print("[JL] 설치하겠습니다.")
    print("[전제조건] MariaDB JL용 계정 및 JL DB 있어야함. ")
    cc.run(jl.install_pacakages())
    cc.run(jl.install_JL())
#def mod_http_dns(web_dir,third_domain):
    mod_http_dns("/home/jl","jl")
    print("[JL] 설정 완료")

#---------------------실행부-----------------------#
#install_JL()


실행


현재 OS를 입력해주세요. ex. Rocky, Ubuntu :  Ubuntu


MariaDB init start
현재 당신의 os는 Ubuntu입니다.
smb 패키지를 설치합니다.
The following packages were automatically installed and are no longer required:
  libutempter0                     linux-modules-extra-6.14.0-15-generic
  linux-headers-6.14.0-15          linux-tools-6.14.0-15
  linux-headers-6.14.0-15-generic  linux-tools-6.14.0-15-generic
  linux-modules-6.14.0-15-generic
Use 'apt autoremove' to remove them.

Installing:
  cifs-utils  expect  samba  smbclient

Installing dependencies:
  attr                  libldb2        libwbclient0    samba-common-bin
  keyutils              libsmbclient0  python3-ldb     samba-libs
  libavahi-client3      libtalloc2     python3-samba   tcl-expect
  libavahi-common-data  libtdb1        python3-talloc  tdb-tools
  libavahi-common3      libtevent0t64  python3-tdb
  libcups2t64           liburing2      samba-common

Suggested packages:
  winbind  cups-common  samba-vfs-ceph       heimdal-clients
  tk8.6    ctdb         samba-vfs-glusterfs

Summary:
  Upgrading

smb 서버와 클라이언트 중 선택하세요.(S/C) ex)S :  C


[SMB] CLIENT 시작


접속할 smb 서버의 ip주소를 입력하세요. ex)172.16.12.100 or 도메인: 172.16.16.25
본인(클라이언트) 서버에 연결할 디렉토리를 지정하세요. ex) /mnt/win_share :  /mnt/win_share
smb id를 입력하세요. ex)smb :  smbb
smb password를 입력하세요. ex)1234 :  1234


SMB 서버에 로그인하겠습니다
spawn smbclient -U smbb -L 172.16.16.25
Password for [WORKGROUP\smbb]:

	Sharename       Type      Comment
	---------       ----      -------
	print$          Disk      Printer Drivers
	share           Disk      smbb's SMB
	IPC$            IPC       IPC Service (Samba 4.22.4)
	smbb            Disk      Home Directories
SMB1 disabled -- no workgroup available



서버에 지정된 공유 폴더의 이름을 입력하세요. ex)share :  share
/etc/fstab에 부팅 시 자동마운트 설정을 하시겠습니까? n 입력시 수동마운트 적용 ex) y/n :  n


Host "172.16.16.25" resolved to the following IP addresses: 172.16.16.25
mount.cifs kernel mount options: ip=172.16.16.25,unc=\\172.16.16.25\share,vers=3.0,user=smbb,pass=********

[SMB] CLIENT 완료
